In [1]:
import os
import cv2
import time
import numpy as np
from tensorflow import keras    
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split

from util.eventvision import read_dataset, read_annotation
from util.BoundingBoxRegressionUtils import makeModel, parse_partial_dataset, parse_random_dataset, tf_iou

TIME_FRAME = 10000
INPUT_SHAPE = keras.Input(shape=(128, 128, 1))

Event-based vision module imported


In [2]:
def makeBB(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    ax = plt.gca()
    
    return Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='red', facecolor='none', label="ground truth")

def getData(filename):
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    arr = np.array(td.data)
    
    matrix = np.zeros((height, width))
    
    for item in arr:
        if item[3] > TIME_FRAME:
            break
        else:
            x = item[0]
            y = item[1]
            matrix[y][x] = item[3] # item[3] = time stamp
            
    matrix = (matrix / TIME_FRAME)**2       
    matrix = cv2.resize(matrix, (128, 128))
    
    return width, height, matrix.reshape([128,128,1])

def getAnnotation(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    return minX, minY, maxX, maxY

In [3]:
def testDataTime():
    data_results = np.empty(20)
    
    for i in range(20):
        data_start_time = time.process_time() 

        X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        data_time = time.process_time() - data_start_time
        
        data_results[i] = data_time
        
        print("Round ", i, " time ", data_time)
    
    return np.mean(data_results)

# print("data avg", testDataTime())

In [4]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def test(model, test_data):
    predictions = model.predict(test_data[0])
    
    bb_scores = tf_iou(predictions[:, -4:], test_data[1][:, -4:])
        
    return np.mean(bb_scores)

def makeTestData():
    testData = []
    
    for i in range(10):
        testData.append(parse_random_dataset(getData, getAnnotation, True))
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_data = testData[i]

        testing_time = time.time()

        iou = test(model, test_data)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = makeModel(INPUT_SHAPE, 8)
    
    train_start_time = time.process_time()

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

Round  0
Training done!  29.203125
	For this mini round, time:  0.5019783973693848  iou  0.78446066
	For this mini round, time:  0.2299652099609375  iou  0.7760176
	For this mini round, time:  0.2330024242401123  iou  0.7847123
	For this mini round, time:  0.22699809074401855  iou  0.7813143
	For this mini round, time:  0.2519998550415039  iou  0.77128357
	For this mini round, time:  0.22302556037902832  iou  0.780628
	For this mini round, time:  0.2210066318511963  iou  0.7794542
	For this mini round, time:  0.22897005081176758  iou  0.77500135
	For this mini round, time:  0.23002409934997559  iou  0.77380764
	For this mini round, time:  0.22800230979919434  iou  0.7783076
Time on avg:  0.2574972629547119  IOU on avg:  0.7784987270832062
Round  1
Training done!  26.578125
	For this mini round, time:  0.28502368927001953  iou  0.78266275
	For this mini round, time:  0.23097729682922363  iou  0.78274566
	For this mini round, time:  0.2230243682861328  iou  0.7862308
	For this mini round

Training done!  26.21875
	For this mini round, time:  0.28202390670776367  iou  0.80635595
	For this mini round, time:  0.22499966621398926  iou  0.8066013
	For this mini round, time:  0.22000360488891602  iou  0.81408125
	For this mini round, time:  0.22697234153747559  iou  0.80872124
	For this mini round, time:  0.22400283813476562  iou  0.8037923
	For this mini round, time:  0.22299718856811523  iou  0.8084549
	For this mini round, time:  0.21899890899658203  iou  0.80595994
	For this mini round, time:  0.22300028800964355  iou  0.807736
	For this mini round, time:  0.2240002155303955  iou  0.80498904
	For this mini round, time:  0.27303123474121094  iou  0.8083129
Time on avg:  0.23400301933288575  IOU on avg:  0.8075004756450653
Round  12
Training done!  26.328125
	For this mini round, time:  0.28800034523010254  iou  0.8190329
	For this mini round, time:  0.22500014305114746  iou  0.8209091
	For this mini round, time:  0.22099995613098145  iou  0.82552516
	For this mini round, t

(array([29.203125, 26.578125, 26.75    , 26.71875 , 26.9375  , 25.953125,
        26.171875, 26.671875, 26.46875 , 26.03125 , 26.890625, 26.21875 ,
        26.328125, 27.078125, 26.6875  , 26.875   , 26.3125  , 27.359375,
        26.421875, 27.      ]),
 array([0.25749726, 0.23400187, 0.22989991, 0.23370183, 0.23359742,
        0.23530037, 0.2314023 , 0.23370295, 0.25629752, 0.22819986,
        0.23970063, 0.23400302, 0.2339    , 0.2372    , 0.23540008,
        0.25990005, 0.22919996, 0.23119984, 0.23059983, 0.23110054]),
 array([0.77849873, 0.78185607, 0.77979361, 0.78555337, 0.79753518,
        0.80912749, 0.81397127, 0.8072154 , 0.79605195, 0.81761028,
        0.80496036, 0.80750048, 0.81981006, 0.80979948, 0.80479531,
        0.77196668, 0.79672979, 0.79132762, 0.81274275, 0.80773123]))

In [3]:
def trainModel():
    model = makeModel(INPUT_SHAPE, 8)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)
    
    return history, model

history, model = trainModel()

NameError: name 'X_train' is not defined

In [6]:
w, h, points = getData(os.getcwd() + r'\..\Caltech101\airplanes\image_0002.bin')

inputPoints = points.reshape([1, 128,128,1])
preds = model.predict(inputPoints)[0]
className = np.argmax(preds[:2])

minX = preds[4] * w
minY = preds[5] * h
maxX = preds[6] * w
maxY = preds[7] * h


points = cv2.resize(points, (w,h))
fig = plt.figure(figsize=(20,10))
plt.imshow(points)
ax = plt.gca()
ax.add_patch(makeBB(os.getcwd() + r'\..\Caltech101_annotations\airplanes\annotation_0002.bin'))
ax.add_patch(Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='green', facecolor='none', label="predicted"))
ax.text((minX + maxX) / 2,(minY - 3),str(className),verticalalignment='top',color='white',fontsize=15,weight='bold')
ax.legend(loc="upper left", prop={'size': 15})

NameError: name 'model' is not defined

In [7]:
from PIL import Image

def generate_gif(save_path, images):
    imgs = []
    for image in images:
        #color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        imgs.append(Image.fromarray(image * 255))

    # duration is the number of milliseconds between frames
    imgs[0].save(save_path, save_all=True, append_images=imgs[1:], duration=100, loop=0)

    
def getDataTimeframe(filename, timeframe, TIME_FRAME=10000):
    """
    Read events from a file stored at filename.
    Return width, height of frame with time frame window
    """
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    arr = np.array(td.data)
    
    matrix = np.zeros((height, width))
    
    for item in arr:
        if item[3] > (timeframe * 10000) and item[3] < (timeframe+1) * 10000:
            x = item[0]
            y = item[1]
            matrix[y][x] = item[3] - (timeframe*10000)
        if item[3] > (timeframe+1)* 10000:
            break
            
    matrix = (matrix / TIME_FRAME)**2       
    matrix = cv2.resize(matrix, (128, 128))
    
    return width, height, matrix.reshape([128,128,1])

def makeGif():
    images = []
    #Somehow make GIF https://www.tutorialspoint.com/how-to-update-matplotlib-s-imshow-window-interactively
    for i in range(30):
        w, h, points = getDataTimeframe(os.getcwd() + r'\..\Caltech101\airplanes\image_0002.bin', i)

        w = 800
        h = 600

        inputPoints = points.reshape([1, 128,128,1])
        preds = model.predict(inputPoints)[0]
        preds[0] = minX = preds[0] * w
        preds[1] = minY = preds[1] * h
        preds[2] = maxX = preds[2] * w
        preds[3] = maxY = preds[3] * h

        points = cv2.resize(points, (w,h))

        cv2.rectangle(points, (int(minX),int(maxY)), (int(maxX), int(minY)), (255,0,0), 2)

        images.append(np.array(points, dtype="float32"))

    generate_gif('./testing.gif', images)
    
# makeGif()

In [8]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

print(np.mean(history.history['val_loss']))
print(np.mean(history.history['val_accuracy']))

NameError: name 'history' is not defined